## Vamos a extrer la información de los diferentes datasets de amazon 

In [9]:
import json
import pandas as pd
from glob import glob


def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(str(v))
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

def convertAllJSON2CSV(): 
    for json_filename in glob('*.json'):
        csv_filename = '%s.csv' % json_filename[:-5]
        print 'Converting %s to %s' % (json_filename, csv_filename)
        df = pd.DataFrame([convert(line) for line in file(json_filename)])
        df.to_csv(csv_filename, encoding='utf-8', index=False)

In [5]:
cachedStopWords

NameError: name 'cachedStopWords' is not defined

In [36]:
# Hay que igualar y coger muestras de tamaño similar para todas las valoraciones?

## 2.-  Procesamiento de texto sobre las reviews
Eliminamos stopwords (TF-idf)<br/>
aplicamos métodos de lematizacion - TextBlob, NLTK <br/>
extraemos entidades (tokens)</br>


In [6]:
from textblob import TextBlob
from textblob import Word
import nltk
# nltk.download() hay que hacerlo la primera vez para cargar todos los corpus necesarios


## funcion que elimina las stopwords y aplica lematizacion

In [7]:
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("english")
cachedStopWords


[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

In [5]:
import string
def remove_punctuation(text):  
    #salida = text.translate(None, string.punctuation).lower()
    salida = text.translate(None, string.punctuation)
    return salida

def lemmatizeReviewText(row):
    words = TextBlob(row["review_clean"]).words.lemmatize()
    ext = ' '.join(word for word in words if word not in (cachedStopWords))
    return ext

In [10]:
babyDf = pd.DataFrame([convert(line) for line in file('reviews_Baby_5.json')])
# forzamos la columna reviewText como str para el correcto funcionamiento de remove_punctuation
babyDf['reviewText'] = babyDf['reviewText'].astype(str)
babyDf['review_clean'] = babyDf['reviewText'].apply(remove_punctuation)
babyDf['review_clean']= babyDf.apply(lemmatizeReviewText, axis=1)
babyDf['review_clean']=babyDf['review_clean'].apply(lambda x: x.lower())
babyDf.to_csv('reviews_Baby_5.csv', header=True,quoting=1)


In [11]:
# Tratamos el dataset de Pets
petsDf = pd.DataFrame([convert(line) for line in file('reviews_Pet_Supplies_5.json')])
# forzamos la columna reviewText como str para el correcto funcionamiento de remove_punctuation
petsDf['reviewText'] = petsDf['reviewText'].astype(str)
petsDf['review_clean'] = petsDf['reviewText'].apply(remove_punctuation)
petsDf['review_clean']= petsDf.apply(lemmatizeReviewText, axis=1)
petsDf['review_clean']= petsDf['review_clean'].apply(lambda x: x.lower())
petsDf.to_csv('reviews_Pet_Supplies_5.csv', header=True,quoting=1)

# Generamos el dataframe amazonReviewsDf a partir de los dataframes  petsDf y babyDf

In [12]:
principalDf = [babyDf,petsDf]
amazonReviewsDf = pd.concat(principalDf)
# renombramos la columna overall por review_overall
amazonReviewsDf['review_overall']=amazonReviewsDf['overall']
del(amazonReviewsDf['overall'])
len(amazonReviewsDf)

318628

# Convertimos el dataFrame al csv que sera la fuente de entrada al modelo de ML

In [13]:
amazonReviewsDf.to_csv('amazon_reviews.csv', header=True,quoting=1)

In [14]:
# comprobamos que está bien generado el csv volviendolo a cargar en el DF
amazonReviewsDf=pd.read_csv('amazon_reviews.csv',header=False,sep=',')
amazonReviewsDf['review_clean'].sample(10)



134853    it spill easily 13 month old grandson might wo...
255821    my older cat got kind eye problem gunk wa comi...
192306    i almost 6 month my cat love fight occasionall...
316742    carpet pet urine odor stain remover amaziing s...
27344     i wa sorting 18 month old daughter toy today i...
311764    my dogpuppy loves antler he chew time the marr...
313932    makes kitty work treat cool took lot time trai...
9792      i didnt like reason they take much room freeze...
154501    there way baby pull i barely pull my husband h...
98081     this worked great son doe like pacifier neithe...
Name: review_clean, dtype: object

In [15]:
#  crear un diccionario {palabra:ocurrencias}
num_important_words = 1500
import numpy as np
cnt = {}
for linea in amazonReviewsDf['review_clean'].astype(str).values:
    for word in linea.split():
        if (word not in cnt):
            cnt[word] = 1
        else:
            cnt[word] += 1
            
wordsSerie=pd.Series(cnt,index=cnt.keys())
wordsSerie=wordsSerie[wordsSerie.values>1]
data={'a':wordsSerie.index,'b':wordsSerie.values}
wordsDf=pd.DataFrame(data=data, index=np.arange(len(wordsSerie)))
wordsDf=wordsDf.sort('b', ascending=False).head(num_important_words)

# Generamos el array de palabras importantes

In [16]:
important_words = str([str(s) for s in wordsDf['a']])
f= open('important_words.json', 'w') 
f.write(important_words.replace("'",'"'))
f.close()
len(wordsDf)

1500

# Procesamiento de us_tweets.json --> us_tweets.csv
conversión de us_tweets_json a us_tweets.csv para aplicar proceso de lematización y eliminación de stopwords

In [741]:
# utilizado para descartar caracteres no ASCII
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

# Devuelve un dataframe a partir del csv de reviews de Tweeter
def getTweetsDF(json_filename):
    csv_filename = '%s.csv' % json_filename[:-5]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    df = pd.DataFrame( columns=['id','text','created_at','geo','city'])

    json_data=open(json_filename).read()
    ob = json.loads(json_data)
    results = ob['results']
    print(len(results))
    for k  in results:

        if (not k['geo']):
            geo = ''
        else:
            idd = k['id_str']
            #preguntamos si el texto está en ASCII para evitar que entren  caracteres UNICODE problematicos en NLP
            if (is_ascii( k['text'])):
                text = k['text']
            else:
                text = ''
            created_at = k['created_at']
            geo= ', '.join(map(str,k['geo']['coordinates']))
            if (is_ascii(k['place']['full_name'])):
                city = k['place']['full_name']
            else:
                city = ''
            SR_row = pd.Series({'id':idd, 'text':text, 'created_at':created_at,'geo':'['+geo+']','city':city},name=len(df))
            df=df.append(SR_row)
    return df

dfTweeter = getTweetsDF('USA_geo.json')

Converting USA_geo.json to USA_geo.csv
9354


# Tratamos el dataset de us_tweets

In [742]:
# lematiza y elimina Stop Words de la fila del dataset de reviews de Tweeter
def lemmatizeReviewTextForTweets(row):
    if (type(row['text']==str)):
        words = TextBlob(row['text']).words.lemmatize()
        ext = ' '.join(word for word in words if word not in (cachedStopWords))
    else:
        ext = ''
    return ext



In [743]:
dfTweeter['text_clean'] = dfTweeter.apply(lemmatizeReviewTextForTweets,axis=1)
dfTweeter['text_clean'] = dfTweeter['text_clean'].astype(str)
dfTweeter['text_clean'] = dfTweeter['text_clean'].apply(remove_punctuation)
# nos quedamos solo con los que tengan informado el campo text_clean
dfTweeter=dfTweeter[dfTweeter['text_clean']!='']
# Generamos us_tweets.csv
dfTweeter.to_csv('us_tweets.csv', header=True,quoting=1)